In [ ]:
!pip install feedparser requests

In [ ]:
import feedparser
import requests
from concurrent.futures import ThreadPoolExecutor
import os
import math

def download_mp3(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {filename}')

def split_mp3(filename, chunk_length=10*60): # default chunk length of 10 minutes
    from pydub import AudioSegment
    audio = AudioSegment.from_mp3(filename)
    chunk_size = chunk_length * 1000 # pydub calculates in millisec
    chunks = [audio[i:i+chunk_size] for i in range(0, len(audio), chunk_size)]
    return chunks

def save_chunks(chunks, base_filename):
    chunk_filenames = []
    for i, chunk in enumerate(chunks):
        chunk_filename = f'{base_filename}_chunk{i}.mp3'
        chunk.export(chunk_filename, format='mp3')
        chunk_filenames.append(chunk_filename)
        print(f'Exported {chunk_filename}')
    return chunk_filenames

def process_feed(url):
    feed = feedparser.parse(url)
    with ThreadPoolExecutor() as executor:
        for entry in feed.entries:
            mp3_url = entry.enclosures[0]['href']
            filename = mp3_url.split('/')[-1]
            if not os.path.exists(filename):
                executor.submit(download_mp3, mp3_url, filename)
                chunks = split_mp3(filename)
                save_chunks(chunks, filename.split('.')[0])

rss_url = 'https://feeds.megaphone.fm/HSW9670766688'
process_feed(rss_url)